In [58]:
from atj_algotrading.backtester import get_ohlc_history, create_price_fig, Backtester, evaluate_backtest
from datetime import datetime, timedelta
import MetaTrader5 as mt5
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [59]:
mt5.initialize()

symbol = 'USTEC'
start_dt = datetime.now() - timedelta(days=365 * 3)
end_dt = datetime.now()

# get historical data
mt5.initialize()
ohlc_d1 = get_ohlc_history(symbol, mt5.TIMEFRAME_D1, start_dt, end_dt)
ohlc_d1['date'] = ohlc_d1['time'].dt.date
ohlc_d1 = ohlc_d1.rename(columns={'high': 'daily_high', 'low': 'daily_low'})

display(ohlc_d1)

,time,open,daily_high,daily_low,close,date
0,2021-10-13,14606.22,14805.06,14602.97,14778.15,2021-10-13
1,2021-10-14,14780.78,15082.67,14778.16,15076.18,2021-10-14
2,2021-10-15,15070.94,15154.41,15032.48,15154.16,2021-10-15
3,2021-10-18,15156.95,15307.38,15050.93,15298.89,2021-10-18
4,2021-10-19,15299.65,15439.42,15281.17,15423.44,2021-10-19
...,...,...,...,...,...,...
771,2024-10-07,20020.38,20065.25,19757.50,19826.33,2024-10-07
772,2024-10-08,19833.58,20134.29,19713.77,20092.44,2024-10-08
773,2024-10-09,20093.19,20286.97,20004.65,20251.09,2024-10-09
774,2024-10-10,20250.93,20317.01,20112.63,20248.53,2024-10-10


In [60]:
mt5.initialize()
ohlc_h1 = get_ohlc_history(symbol, mt5.TIMEFRAME_H1, start_dt, end_dt)
ohlc_h1['date'] = ohlc_h1['time'].dt.date

ohlc_max_values = ohlc_h1.sort_values(['date', 'high'], ascending=[True, False])[['time', 'date', 'high']]
ohlc_max_values2 = ohlc_max_values.groupby('date').first()
ohlc_max_values2['type'] = 'high'
ohlc_max_values2 = ohlc_max_values2.rename(columns={'high': 'value'})

ohlc_min_values = ohlc_h1.sort_values(['date', 'low'], ascending=[True, True])[['time', 'date', 'low']]
ohlc_min_values2 = ohlc_min_values.groupby('date').first()
ohlc_min_values2['type'] = 'low'
ohlc_min_values2 = ohlc_min_values2.rename(columns={'low': 'value'})

display(ohlc_max_values2)
display(ohlc_min_values2)

min_max_df = pd.concat([ohlc_max_values2, ohlc_min_values2])
min_max_df = min_max_df.sort_values('time')
display(min_max_df)

px.line(min_max_df, x='time', y='value')

,time,value,type
date,,,
2021-10-12,2021-10-12 15:00:00,14794.98,high
2021-10-13,2021-10-13 21:00:00,14805.06,high
2021-10-14,2021-10-14 23:00:00,15082.67,high
2021-10-15,2021-10-15 23:00:00,15154.41,high
2021-10-18,2021-10-18 23:00:00,15307.38,high
...,...,...,...
2024-10-07,2024-10-07 01:00:00,20065.25,high
2024-10-08,2024-10-08 22:00:00,20134.29,high
2024-10-09,2024-10-09 22:00:00,20286.97,high


,time,value,type
date,,,
2021-10-12,2021-10-12 23:00:00,14597.21,low
2021-10-13,2021-10-13 01:00:00,14602.97,low
2021-10-14,2021-10-14 01:00:00,14778.16,low
2021-10-15,2021-10-15 04:00:00,15032.48,low
2021-10-18,2021-10-18 10:00:00,15050.93,low
...,...,...,...
2024-10-07,2024-10-07 22:00:00,19757.50,low
2024-10-08,2024-10-08 01:00:00,19713.77,low
2024-10-09,2024-10-09 10:00:00,20004.65,low


,time,value,type
date,,,
2021-10-12,2021-10-12 15:00:00,14794.98,high
2021-10-12,2021-10-12 23:00:00,14597.21,low
2021-10-13,2021-10-13 01:00:00,14602.97,low
2021-10-13,2021-10-13 21:00:00,14805.06,high
2021-10-14,2021-10-14 01:00:00,14778.16,low
...,...,...,...
2024-10-09,2024-10-09 22:00:00,20286.97,high
2024-10-10,2024-10-10 16:00:00,20112.63,low
2024-10-10,2024-10-10 19:00:00,20317.01,high


In [61]:
min_max_df2 = min_max_df.copy()

min_max_df2['prev_type'] = min_max_df2['type'].shift(1)
min_max_df2['prev_value'] = min_max_df2['value'].shift(1)
min_max_df2['prev_time'] = min_max_df2['time'].shift(1)
min_max_df2['same_type_in_a_row'] = min_max_df2.apply(lambda x: 1 if x['type'] == x['prev_type'] else 0, axis=1)

def choose_min_max_value(x):
    if x['same_type_in_a_row'] == 1 and x['type'] == 'low':
        if x['value'] < x['prev_value']:
            return x['value'], x['time']
        else:
            return x['prev_value'], x['prev_time']
        
    elif x['same_type_in_a_row'] == 1 and x['type'] == 'high':
        if x['value'] > x['prev_value']:
            return x['value'], x['time']
        else:
            return x['prev_value'], x['prev_time']
    
min_max_df2['new_data'] = min_max_df2.apply(choose_min_max_value, axis=1)
min_max_df2['new_data'] = min_max_df2['new_data'].shift(-1)

def overwrite_value(x):
    if x['new_data']:
        return x['new_data'][0]
    else:
        return x['value']
    
def overwrite_time(x):
    if x['new_data']:
        return x['new_data'][1]
    else:
        return x['time']

min_max_df2['time'] = min_max_df2.apply(overwrite_time, axis=1)
min_max_df2['value'] = min_max_df2.apply(overwrite_value, axis=1)

min_max_df2 = min_max_df2[min_max_df2['same_type_in_a_row'] == 0]
min_max_df2

,time,value,type,prev_type,prev_value,prev_time,same_type_in_a_row,new_data
date,,,,,,,,
2021-10-12,2021-10-12 15:00:00,14794.98,high,None,NaN,NaT,0,None
2021-10-12,2021-10-12 23:00:00,14597.21,low,high,14794.98,2021-10-12 15:00:00,0,"(14597.21, 2021-10-12 23:00:00)"
2021-10-13,2021-10-13 21:00:00,14805.06,high,low,14602.97,2021-10-13 01:00:00,0,None
2021-10-14,2021-10-14 01:00:00,14778.16,low,high,14805.06,2021-10-13 21:00:00,0,None
2021-10-14,2021-10-14 23:00:00,15082.67,high,low,14778.16,2021-10-14 01:00:00,0,None
...,...,...,...,...,...,...,...,...
2024-10-09,2024-10-09 10:00:00,20004.65,low,high,20134.29,2024-10-08 22:00:00,0,None
2024-10-09,2024-10-09 22:00:00,20286.97,high,low,20004.65,2024-10-09 10:00:00,0,None
2024-10-10,2024-10-10 16:00:00,20112.63,low,high,20286.97,2024-10-09 22:00:00,0,None


In [62]:
fig = create_price_fig(ohlc_h1)
fig.add_trace(go.Scatter(x=min_max_df2['time'], y=min_max_df2['value'], mode="lines"))

fig.show()

In [71]:
hl_df = min_max_df2.copy()[['time', 'value', 'type']]
hl_df['recent_hls'] = hl_df['type'].rolling(4).apply(lambda x: ','.join(x))
hl_df

DataError: No numeric types to aggregate